# Binary Classification Using the ChestX-ray14 Dataset

## Step 0: Install PyHealth

In [ ]:
%pip install pyhealth ipywidgets

## Step 1: Load Dataset

In [2]:
from pyhealth.datasets import ChestXray14Dataset

dataset = ChestXray14Dataset(download=True, partial=True)
dataset.stats()

Checking MD5 checksum for ./images_01.tar.gz...
Extracting ./images_01.tar.gz...
Deleting ./images_01.tar.gz...
Download complete
Initializing ChestX-ray14 dataset from . (dev mode: False)
No cache_dir provided. Using default cache dir: /root/.cache/pyhealth/fb6e8a46-32a1-580b-bb6c-4015d54b1bc1
Scanning table: chestxray14 from /root/chestxray14-metadata-pyhealth.csv
Caching event dataframe to /root/.cache/pyhealth/fb6e8a46-32a1-580b-bb6c-4015d54b1bc1/global_event_df.parquet...
Dataset: ChestX-ray14
Dev mode: False
Number of patients: 1335
Number of events: 4999


## Step 2: Define Task

In [3]:
from pyhealth.tasks import ChestXray14BinaryClassification

task = ChestXray14BinaryClassification(disease="infiltration")
samples = dataset.set_task(task)

Setting task ChestXray14BinaryClassification for ChestX-ray14 base dataset...
Applying task transformations on data with 1 workers...
Detected Jupyter notebook environment, setting num_workers to 1
Single worker mode, processing sequentially
Worker 0 started processing 1335 patients. (Polars threads: 22)


  0%|          | 0/1335 [00:00<?, ?it/s]

Rank 0 inferred the following `['bytes']` data format.


100%|██████████| 1335/1335 [00:04<00:00, 311.17it/s]

Worker 0 finished processing patients.
Fitting processors on the dataset...


...

  0%|          | 0/4999 [00:00<?, ?it/s]

Rank 0 inferred the following `['tensor', 'no_header_tensor:1']` data format.


100%|██████████| 4999/4999 [01:21<00:00, 61.52it/s]

Worker 0 finished processing samples.


Cached processed samples to /root/.cache/pyhealth/fb6e8a46-32a1-580b-bb6c-4015d54b1bc1/tasks/ChestXray14BinaryClassification_acac6c08-a7e0-5016-99fb-ace461d83f56/samples_0a9d8d5e-42c4-534f-9f35-24cffedeb0db.ld


In [4]:
from pyhealth.datasets import get_dataloader, split_by_sample

train_dataset, val_dataset, test_dataset = split_by_sample(samples, [0.7, 0.1, 0.2])

train_loader = get_dataloader(train_dataset, batch_size=16, shuffle=True)
val_loader = get_dataloader(val_dataset, batch_size=16, shuffle=False)
test_loader = get_dataloader(test_dataset, batch_size=16, shuffle=False)

## Step 3: Define Model

In [5]:
from pyhealth.models import CNN

model = CNN(dataset=samples)

/usr/local/lib/python3.13/site-packages/pyhealth/sampler/sage_sampler.py:3: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


## Step 4: Train Model

In [6]:
from pyhealth.trainer import Trainer

trainer = Trainer(model=model)
trainer.train(train_dataloader=train_loader, val_dataloader=val_loader, epochs=1)

CNN(
  (embedding_model): EmbeddingModel(embedding_layers=ModuleDict())
  (cnn): ModuleDict(
    (image): CNNLayer(
      (cnn): ModuleList(
        (0): CNNBlock(
          (conv1): Sequential(
            (0): Conv2d(1, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU()
          )
          (conv2): Sequential(
            (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          )
          (downsample): Sequential(
            (0): Conv2d(1, 128, kernel_size=(1, 1), stride=(1, 1))
            (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          )
          (relu): ReLU()
        )
      )
      (pooling): AdaptiveAvgPool2d(output_size=1)
    )
  )
  (fc): Linear(in_features=128, out_features=1

Epoch 0 / 1:   0%|          | 0/219 [00:00<?, ?it/s]

--- Train epoch-0, step-219 ---
loss: 0.4468


Evaluation: 100%|██████████| 32/32 [00:24<00:00,  1.32it/s]

--- Eval epoch-0, step-219 ---
pr_auc: 0.2070
roc_auc: 0.6026
f1: 0.0000
loss: 0.4438


## Step 5: Evaluate Model

In [7]:
trainer.evaluate(test_loader)

Evaluation: 100%|██████████| 63/63 [00:47<00:00,  1.31it/s]


{'pr_auc': 0.2557703830457655,
 'roc_auc': 0.6404010464501405,
 'f1': 0.0,
 'loss': 0.44653598089066765}